In [1]:
import yaml
import time
from datetime import datetime

import sys
sys.path.append('../scripts')  # Adjust the path as needed to point to the directory containing utils.py

from alert_utils import classifier, set_device, check_emails, extract_update_camera_status
from megadetector.detection import run_detector
from megadetector.visualization import visualization_utils as vis_utils

In [2]:
# Load settings from configuration file
with open('../config.yaml') as file:
    config = yaml.safe_load(file)

IMAP_HOST = config['imap_config']['host']
EMAIL_USER = config['imap_config']['user']
EMAIL_PASS = config['imap_config']['password']
TELEGRAM_BOT_TOKEN = config['telegram_config']['bot_token']
TELEGRAM_CHAT_ID = '-1002249589791' # replace with config after tests

# Detection and Classification Model Settings
MODEL_PATH_DETECTOR = '../models/md_v5a.0.0.pt'
MODEL_PATH_CLASSIFIER = '../models/deepfaune-vit_large_patch14_dinov2.lvd142m.pt'
BACKBONE = 'vit_large_patch14_dinov2'
ANIMAL_CLASSES = ["badger", "ibex", "red deer", "chamois", "cat", "goat", "roe deer", "dog", "squirrel", "equid", "genet", "hedgehog", "lagomorph", "wolf", "lynx", "marmot", "micromammal", "mouflon", "sheep", "mustelid", "bird", "bear", "nutria", "fox", "wild boar", "cow"]
DETECTOR_CLASSES = ["animal", "human", "vehicle"]
species_of_interest = {"wild boar", "bear", "wolf", "roe deer", "red deer"}
DETECTION_THRESHOLD = 0.05
CLASSIFICATION_THRESHOLD = 0.05

# Load the MegaDetector model
device = set_device()
detector = run_detector.load_detector(MODEL_PATH_DETECTOR)
print("Loading classifier...")
start_time = time.time()
classifier = classifier(MODEL_PATH_CLASSIFIER, BACKBONE, ANIMAL_CLASSES, device)
end_time = time.time()
print(f"Loaded classifier in {(end_time - start_time):.2f} seconds")

Imported YOLOv5 as utils.*
Using PyTorch version 2.3.0+cu121


Fusing layers... 
Fusing layers... 
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs


Sending model to GPU
Loaded model in 1.41 seconds
Loading classifier...
Loaded classifier in 3.27 seconds


In [6]:
if __name__ == "__main__":
    print(f"Monitoring {EMAIL_USER} for new messages...")
    while True:
        try:
            images, camera_id, temp_deg_c, img_date, img_time, battery, sd_memory = \
                check_emails(IMAP_HOST, EMAIL_USER, EMAIL_PASS)
            if camera_id:
                camera_make, gps, location, map_url, battery, sd_memory = extract_update_camera_status(camera_id, battery, sd_memory)
                print(
                    f"\nEmail Received."
                    f"\nImages: {len(images)}, Camera ID: {camera_id}, Camera Make: {camera_make}"
                    f"\nDate: {img_date}, Time: {img_time}, Temperature: {temp_deg_c}"
                    f"\nBattery: {battery}%, SD Memory: {sd_memory}%"
                    f"\nLocation: {location}, GPS: {gps}, Map URL: {map_url}"
                )
                if images:
                    time.sleep(1)
                    # detect_objects()
                    # classify_objects()
                    # annotate_images()
                    # generate_caption()
                    # send_alert()
                    # save_images()
                    # update_excel()
            else:
                time.sleep(1)

        except KeyboardInterrupt:
            print("Interrupted by user")
            break
        #except Exception as e:
        #    print(f"An error occurred: {e}")
        #    print(f"\nMonitoring {EMAIL_USER} for new messages...")
        #    continue


Monitoring fcccameratraps@gmail.com for new messages...

Email Received.
Images: 1, Camera ID: CC044C, Camera Make: UOVision
Date: 2024-05-31, Time: 17:49:54, Temperature: 20
Battery: 100%, SD Memory: 98%
Location: Piscul Lat, GPS: X: 25.01059634, Y: 45.326467, Z: 861m, Map URL: https://maps.app.goo.gl/3g6QH2rn6Qw7Aema6

Email Received.
Images: 1, Camera ID: WILDBOAR02, Camera Make: Wilsus
Date: 2024-05-31, Time: 17:02:24, Temperature: None
Battery: 50%%, SD Memory: 50%%
Location: Piscul Lat, GPS: X: 25.01059634, Y: 45.326467, Z: 862m, Map URL: https://maps.app.goo.gl/3g6QH2rn6Qw7Aema6

Email Received.
Images: 0, Camera ID: WILDBOAR02, Camera Make: Wilsus
Date: 2024-05-31, Time: 17:02:09, Temperature: 22
Battery: 100%, SD Memory: 99%
Location: Piscul Lat, GPS: X: 25.01059634, Y: 45.326467, Z: 862m, Map URL: https://maps.app.goo.gl/3g6QH2rn6Qw7Aema6
Interrupted by user
